In [2]:
import pandas as pd

In [8]:
file = 'legacy_data/20180810 Erfassung Staatsratsprotokolle foto (1).xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
df = pd.read_excel(file, dtype={'Karton': str}).fillna('False')
archiv, _ = Institution.objects.get_or_create(
    written_name='Österreichisches Staatsarchiv / Archiv der Republik',
    abbreviation="ÖStA/AdR",
    institution_type="Archiv"
)

In [9]:
df

,Teil von Bestand,(Archiv)Signatur,Karton,Betreff - Titel des Dokuments,Typ des Dokuments,Datum orig,Datum normalisiert,Im Dokument erwähnte Person,Im Dokument erwähnte Institution,Im Dokument vorkommende Orte,Schlagwörter normalisiert,Verweis,Zusammenfassung,Kommentar,Erzeuger des Dokuments (Person),Erzeuger des Dokuments (Institution)
0,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Nachrichten aus Deutschböhmen,Protokoll,18. Nov. 1918,1918-11-18 00:00:00,Karl Renner; Karl Urban; Franz Dinghofer,Arbeiter-Zeitung; Staatsrat tschechoslowak. Re...,Reichenberg (Liberec); Deutschböhmen; Bayern; ...,besetzes Gebiet; Administration; Geldmittel; G...,False,Bericht über die Situation in Deutschböhmen: e...,False,False,Staatsrat
1,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Verwaltung und Verwahrung der Krongüter,Protokoll,18. Nov. 1918,1918-11-18 00:00:00,Julius Sylvester; Franz Dinghofer,StA für Landwirtschaft; Staatsnotariat,False,"Krongüter, Sachverhalte aus der Monarchie; Ver...",Staatsratsprotokoll Nr. 35 vom 13. November 1918,"Es wird beschlossen, die Verwahrung und Verwal...",False,False,Staatsrat
2,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",False,2,Aufhebung des Kriegsleistungsgesetzes,Protokoll,18. Nov. 1918,1918-11-18 00:00:00,Karl Renner,False,False,"Kriegsleistungsgesetz (Aufhebung), Verwaltung","Kriegsleistungsgesetz, RGBl. 236/1912; Vollzug...","StK Renner plädiert dafür, das Kriegsleistungs...","Das Kriegsleistungsgesetz aus dem Jahre 1912, ...",False,Staatsrat
3,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Bevollmächtigte der Zentralregierung bei den L...,Protokoll,19. Nov. 1918,1918-11-19 00:00:00,Karl Renner; Wilhelm Ellenbogen; Jodok Fink; J...,Landesregierung; deutschösterr. Regierung; StA...,Tirol; Südtirol (Alto Adige); Vorarlberg; Schw...,Bundesländer; Autonomie; Demokratisierung; Bev...,Gesetz vom 14. November 1918 betreffend die Üb...,"StK Renner stellt den Antrag, die Staatsregier...",False,False,False
4,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Übernahme der deutschen Beamten des k.u.k. Min...,False,False,False,Julius Sylvester; Viktor Waldner,k.u.k. Ministerium der Finanzen; Oberster Rech...,False,Staatsbediensteter; deutsche Nationalität; Liq...,False,Staatsnotar Sylvester stellt den dringenden An...,False,False,False
5,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,In Südmähren verhaftete Gendarmen und Offiziere,Protokoll,19. Nov. 1918,1918-11-19 00:00:00,Oskar Teufel; Otto Bauer; Vlastimil Tusar,Tschechoslowakischen Republik (Československa ...,Südmähren (Jizni Morava); Tschechoslowakei (Če...,"Gendarm; Offizier, Verhaftung; deutsche Nation...",False,"Staatsrat Teufel begehrt Aufklärung, welche Ma...",False,False,Staatsrat
6,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Verhaftung eines tschechischen Statthaltereise...,Protokoll,19. Nov. 1918,1918-11-19 00:00:00,Egon Pflügl; Zierhut; Vlastimil Tusar; Schumpe...,Bezirkshauptmannschaft Kapplitz (Kaplice); Sta...,Tschechoslowakei (Československo); Kaplitz (Ka...,umstrittenes Gebiet; Verhaftung; Statthalterei...,False,In Kaplitz (Kaplice) wurde ein tschechischer S...,False,False,Staatsrat
7,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...",2,Antrag auf Aufhebung des Kriegsleistungsgesetzes,Protokoll,19. Nov. 1918,1918-11-19 00:00:00,False,Heereskommission,False,"Kriegsleistungsgesetz (Aufhebung),",Vollzugsanweisung des Staatsamtes für Heereswe...,Ein Antrag betreffend Aufhebung des Kriegsleis...,Tatsächlich wird das Kriegsleistungsgesetz ers...,False,Staatsrat
8,"ÖStA/AdR, Bundeskanzleramt, Staatsratsprotokol...","ÖStA/AdR, Bundeskanzler

In [12]:
for i, row in df.iterrows():
    item = None
    signatur = ",".join(
            [
                "Bestand: {}".format(format(str(row.get('(Archiv)Signatur', default="-")))),
                "Karton: {}".format(str(row.get('Karton', default="-"))), 
                "[internal-id]: {}".format(i)
            ]
    )
    try:
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
    except Exception as e:
        print(e)
    if item:           
        if row['Betreff - Titel des Dokuments'] != 'False':
            item.title = row['Betreff - Titel des Dokuments']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Kommentar'] != 'False':
            item.notes = row['Kommentar']
        if row['Datum orig'] != 'False':
            item.written_date = row['Datum orig']
        if row['Typ des Dokuments'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ des Dokuments'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Datum normalisiert'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert'])
            except Exception as e:
                print(e)
        if row['Im Dokument vorkommende Orte'] != 'False':
            for x in row['Im Dokument vorkommende Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    try:
                        other_name = x.strip().split('(')[0]
                        pl = Place.objects.get(name=other_name)
                    except:
                        pl = None
                        print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Im Dokument erwähnte Institution'] != 'False':
            for x in row['Im Dokument erwähnte Institution'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Im Dokument erwähnte Person'] != 'False':
            for x in row['Im Dokument erwähnte Person'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
        if row['Erzeuger des Dokuments (Institution)'] != 'False':
            for x in row['Erzeuger des Dokuments (Institution)'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.creator_inst.add(pl)
            item.archiv = archiv
            item.save()

Institution matching query does not exist. Arbeiter-Zeitung
Institution matching query does not exist. Staatsrat tschechoslowak. Regierung
Institution matching query does not exist. Buschtehraderbahn
Place matching query does not exist. Tirol
Institution matching query does not exist. Tschechoslowakei (Ceskooslovakia)
Institution matching query does not exist. Prov. Nationalversammlung
Person matching query does not exist. Jodok Fink
Institution matching query does not exist. k.u.k. Ministerium der Finanzen
Institution matching query does not exist. Oberster Rechnungshof
Institution matching query does not exist. Tschechoslowakischen Republik (Československa Republika)
Institution matching query does not exist. Bezirkshauptmannschaft Kapplitz (Kaplice)
Institution matching query does not exist. Statthalterei
Person matching query does not exist. Schumpeter [Konsul]
Institution matching query does not exist. Heereskommission
Institution matching query does not exist. k.u.k. Kriegsminist

Person matching query does not exist. Dr. Wotawa
Person matching query does not exist. Daun
Institution matching query does not exist. deutschösterreichische Staatskanzlei
Institution matching query does not exist. Staatsrat, deutschösterreichische Prov. Nationalversammlung, ehemalige Burggendarmerie
Institution matching query does not exist. Parlamentswache
Person matching query does not exist. Kaiser Karl I.
Person matching query does not exist. Graf Sternberg
Place matching query does not exist. besetztes Gebiet
Place matching query does not exist. Sachsen, Rumburg (Rumburk)
Institution matching query does not exist. Landesregierung Deutschböhmen
Institution matching query does not exist. deutschösterreichische Prov. Nationalversammlung
Institution matching query does not exist. Tschechoslowakei
Institution matching query does not exist. tschechisches Militär, Narodní Výbor
Institution matching query does not exist. k.u.k. Ministeriums des Äußern, Archiv
Institution matching query d

Institution matching query does not exist. Prov. Nationalversammlung (Staatsangestelltenausschuss)
Institution matching query does not exist. Konst. Nationalversammlung
Institution matching query does not exist. Obersthofmarschallamt, Gericht
Institution matching query does not exist. Konst. Nationalversammlung
Place matching query does not exist. Tschecholowakei
Place matching query does not exist. Jugoslawien
Institution matching query does not exist. Kabinettsrat
Institution matching query does not exist. Staatsrat, tschechischer Staat
Institution matching query does not exist. jugoslawischer Staat
Institution matching query does not exist. Sudentenland
Institution matching query does not exist. Jugoslawien
Institution matching query does not exist. ehemals österr.-ungar. Monarchie
Institution matching query does not exist. Deutschösterreich
Institution matching query does not exist. StA für Heereswesen, Volkswehr
Person matching query does not exist. 
Institution matching query doe